# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf

from hot_dust import preprocess, model

2024-01-23 14:52:02.880162: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 14:52:02.920652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 14:52:02.920688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 14:52:02.921370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 14:52:02.926838: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 14:52:02.927283: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 500  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizati  (None, 9)                 19        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 372 (1.46 KB)
Trainable params: 353 (1.38 KB)
Non-trainable params: 19 (80.00 Byte)
_________________________________________________________________


### Fitting

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/500
3122/3122 - 4s - loss: 0.6906 - val_loss: 0.4370 - 4s/epoch - 1ms/step
Epoch 2/500
3122/3122 - 3s - loss: 0.3703 - val_loss: 0.3339 - 3s/epoch - 1ms/step
Epoch 3/500
3122/3122 - 3s - loss: 0.3120 - val_loss: 0.2986 - 3s/epoch - 1ms/step
Epoch 4/500
3122/3122 - 3s - loss: 0.2840 - val_loss: 0.2756 - 3s/epoch - 996us/step
Epoch 5/500
3122/3122 - 3s - loss: 0.2644 - val_loss: 0.2582 - 3s/epoch - 981us/step
Epoch 6/500
3122/3122 - 3s - loss: 0.2490 - val_loss: 0.2442 - 3s/epoch - 978us/step
Epoch 7/500
3122/3122 - 3s - loss: 0.2368 - val_loss: 0.2336 - 3s/epoch - 977us/step
Epoch 8/500
3122/3122 - 3s - loss: 0.2275 - val_loss: 0.2255 - 3s/epoch - 994us/step
Epoch 9/500
3122/3122 - 3s - loss: 0.2199 - val_loss: 0.2179 - 3s/epoch - 989us/step
Epoch 10/500
3122/3122 - 3s - loss: 0.2131 - val_loss: 0.2117 - 3s/epoch - 987us/step
Epoch 11/500
3122/3122 - 3s - loss: 0.2075 - val_loss: 0.2061 - 3s/epoch - 997us/step
Epoch 12/500
3122/3122 - 3s - loss: 0.2023 - val_loss: 0.2007 - 3s/ep

INFO:tensorflow:Assets written to: data/network/assets


### Training and Validation Losses

In [8]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)